In [1]:
import sys
import os

#otherwise our imports get all messed up
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import pandas as pd

from K_mean_cluster import run_clustering




In [3]:
#BA_US_knn_text = pd.read_csv('../../../knnData/BA_US_knn_text.csv')

#run_clustering(BA_US_knn_text)

In [4]:
#RB_US_knn_text = pd.read_csv('../../../knnData/RB_US_knn_text.csv')

#run_clustering(RB_US_knn_text)

In [5]:
#UMAP clustering
#how do i import it from prepData
from utils.prepData import prep_data
BA_US_knn_text = pd.read_csv('../../../knnData/BA_US_knn_text.csv')
df_total_clustering, states = prep_data(BA_US_knn_text)


In [6]:
import umap
from mpl_toolkits.mplot3d import Axes3D

df_total_clustering_cleaned = df_total_clustering.drop(columns=['user_state'])

#print(df_total_clustering_cleaned.shape)
#df_cleaned = df_total_clustering_cleaned.fillna(0)
#print(df_cleaned.shape)

df_cleaned = df_total_clustering_cleaned.fillna(df_total_clustering_cleaned.mean())
print(df_cleaned.shape)
df_cleaned = df_cleaned.dropna(axis=1) #for some reason theres one row in RB thats NaNs, so we drop it
print(df_cleaned.shape)

umap_3d = umap.UMAP(n_components=3, random_state=42)
embedding_3d = umap_3d.fit_transform(df_cleaned)


(50, 824)
(50, 824)


c:\Users\Enzo\anaconda3\envs\ada\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [7]:
import umap
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

# Step 1: Drop the 'user_state' column as it's just an identifier and not used for UMAP
df_total_clustering_cleaned = df_total_clustering.drop(columns=['user_state'])

print(df_total_clustering_cleaned.shape)
# Fill NaN values with 0
df_cleaned = df_total_clustering_cleaned.fillna(0)
# Assuming df_total_clustering_cleaned is the cleaned dataframe (no 'user_state')
# Apply UMAP to reduce dimensions to 3
umap_3d = umap.UMAP(n_components=3, random_state=42)
embedding_3d = umap_3d.fit_transform(df_cleaned)

# Perform KMeans clustering with 4 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(embedding_3d)

# Add the cluster labels back to the dataframe
df_cleaned['Cluster'] = kmeans_labels
df_cleaned['user_state'] = df_total_clustering['user_state']  # Assuming original_df has 'user_state'

#Plot the 3D UMAP with Plotly and only show 'user_state' on hover
fig = px.scatter_3d(df_cleaned, x=embedding_3d[:, 0], y=embedding_3d[:, 1], z=embedding_3d[:, 2],
                    color='Cluster', title="K-means Clustering on UMAP",
                    labels={'Cluster': 'Cluster Label'},
                    hover_data={'user_state': True, 'Cluster': False})  # Only show user_state on hover, hide Cluster

fig.show()

# Output states in each cluster
clustered_states = {}
for cluster_num in range(4):
    states_in_cluster = df_cleaned[df_cleaned['Cluster'] == cluster_num]['user_state'].tolist()
    clustered_states[cluster_num] = states_in_cluster

# Print out the states in each cluster
for cluster_num, states in clustered_states.items():
    print(f"Cluster {cluster_num}: {states}")


(50, 824)


c:\Users\Enzo\anaconda3\envs\ada\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\Enzo\anaconda3\envs\ada\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Cluster 0: ['Alaska', 'Delaware', 'Maine', 'New Hampshire', 'Texas', 'West Virginia']
Cluster 1: ['California', 'Colorado', 'Florida', 'Illinois', 'Indiana', 'Kentucky', 'Maryland', 'Massachusetts', 'Missouri', 'New Jersey', 'New York', 'Ohio', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'Vermont', 'Washington']
Cluster 2: ['Connecticut', 'Georgia', 'Michigan', 'Minnesota', 'North Carolina', 'Oregon', 'Tennessee', 'Virginia', 'Wisconsin']
Cluster 3: ['Alabama', 'Arizona', 'Arkansas', 'Hawaii', 'Idaho', 'Iowa', 'Kansas', 'Louisiana', 'Mississippi', 'Montana', 'Nebraska', 'Nevada', 'New Mexico', 'North Dakota', 'Oklahoma', 'South Dakota', 'Utah', 'Wyoming']


In [8]:
from utils.map import draw_map
draw_map(clustered_states)

  state_name    group state_abbreviation
0   Colorado  Group 1                 CO
1  Louisiana  Group 3                 LA


In [9]:
from sklearn.cluster import DBSCAN
# Step 2: Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.01, min_samples=5)  # You can tune these parameters
dbscan_labels = dbscan.fit_predict(embedding_3d)

# Step 3: Create a DataFrame with the DBSCAN labels and original states (if needed)
df_clustering = df_total_clustering.copy()  # Assuming 'user_state' is part of df_total_clustering
df_clustering['DBSCAN_Cluster'] = dbscan_labels

# Output the states in each cluster
for cluster in set(dbscan_labels):
    print(f"Cluster {cluster}:")
    print(df_clustering[df_clustering['DBSCAN_Cluster'] == cluster]['user_state'].tolist())

# Step 4: Visualize the clustering result in 3D with Plotly
fig = px.scatter_3d(df_clustering, x=embedding_3d[:, 0], y=embedding_3d[:, 1], z=embedding_3d[:, 2],
                    color='DBSCAN_Cluster', title="DBSCAN Clustering on UMAP-reduced Data")

fig.show()

Cluster -1:
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
